## Bibliotecas

In [1]:
import pandas as pd
import numpy as np

## Dados

In [3]:
# Carrega FILMES
filmes = pd.read_csv('dados/movies.csv')
filmes.columns = ['filmeId', 'titulo', 'generos']
filmes = filmes.set_index('filmeId')

# Carrega NOTAS
notas = pd.read_csv('dados/ratings.csv')
notas.columns = ['usuarioId', 'filmeId', 'nota', 'momento']


## Funções

In [4]:
# Retorna as notas com base num Id de usuário
def notas_do_usuario(usuario):
  notas_do_usuario = notas.query(f'usuarioId=={usuario}')
  notas_do_usuario = notas_do_usuario[['filmeId', 'nota']].set_index('filmeId')
  return notas_do_usuario

# Distância de 2 vetores com base numa função do numpy
def distancia_de_vetores(a,b):
  return np.linalg.norm(a - b)

# Distância entre 2 usuários com no mínimo 5 filmes em comum
def dist_de_usuarios(usuario_id1, usuario_id2):
  notas1 = notas_do_usuario(usuario_id1)
  notas2 = notas_do_usuario(usuario_id2)
  diferencas = notas1.join(notas2, lsuffix='_esquerda', rsuffix='_direita').dropna()

  minimo = 5
  if(len(diferencas)< minimo):
    return None

  distancia = distancia_de_vetores(diferencas['nota_esquerda'], diferencas['nota_direita'])
  return [usuario_id1, usuario_id2, distancia]

# Distância de um usuário em relação a todos os outros
def distancia_de_todos(voce_id, n_usuarios_a_analisar = None):
  todos_os_usuarios = notas['usuarioId'].unique()
  if n_usuarios_a_analisar:
    todos_os_usuarios = todos_os_usuarios[:n_usuarios_a_analisar]
  distancias = [dist_de_usuarios(voce_id, usuario_id) for usuario_id in todos_os_usuarios]
  distancias = list(filter(None, distancias))
  distancias = pd.DataFrame(distancias, columns = ['voce', 'outra_pessoa', 'distancia'])
  return distancias

# knn (K nearest neighbors)
def knn(voce_id, k_mais_proximos = 10, n_usuarios_a_analisar = None):
  distancias = distancia_de_todos(voce_id, n_usuarios_a_analisar)
  distancias = distancias.sort_values('distancia')
  distancias = distancias.set_index('outra_pessoa').drop(voce_id, errors='ignore')
  return distancias.head(k_mais_proximos)

# Sugere com base nos KNN do usuário inserido
def sugere_para( voce, k_mais_proximos = 10, n_usuarios_a_analisar = None):
  notas_de_voce = notas_do_usuario(voce)
  filmes_que_voce_ja_viu = notas_de_voce.index

  similares = knn(voce, k_mais_proximos = k_mais_proximos, n_usuarios_a_analisar = n_usuarios_a_analisar)
  usuarios_similares = similares.index
  notas_dos_similares = notas.set_index("usuarioId").loc[usuarios_similares]
  recomendacoes = notas_dos_similares.groupby('filmeId').mean()[['nota']].sort_values("nota", ascending=False)
  aparicoes = notas_dos_similares.groupby('filmeId').count()[['nota']]

  filtro_minimo = k_mais_proximos / 2
  recomendacoes = recomendacoes.join(aparicoes, lsuffix='_media_dos_usuarios', rsuffix = '_aparicoes_nos_usuarios')
  recomendacoes = recomendacoes.query('nota_aparicoes_nos_usuarios >= %.2f' % filtro_minimo)
  recomendacoes = recomendacoes.drop(filmes_que_voce_ja_viu, errors='ignore')
  
  return recomendacoes.join(filmes)


## Testando

In [5]:
sugere_para(1)

,nota_media_dos_usuarios,nota_aparicoes_nos_usuarios,titulo,generos
filmeId,,,,
58559,4.700000,5,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
7153,4.700000,5,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
4993,4.666667,6,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
318,4.583333,6,"Shawshank Redemption, The (1994)",Crime|Drama
109487,4.583333,6,Interstellar (2014),Sci-Fi|IMAX
79132,4.071429,7,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX


In [10]:
# Testando um usuário novo
usuario_novo_dados = [[1,3],[7,4],[28,5],[10,4],[19,4],[8,4],[6,4],[5,4]]

# Cria um novo usuário com base nos dados de filmes que ele já assistiu
def novo_usuario (dados):
  novo_usuario = notas['usuarioId'].max()+1
  display(f'O Id do novo usuário é: {novo_usuario}')
  notas_do_usuario_novo = pd.DataFrame(dados, columns= ['filmeId', 'nota'])
  notas_do_usuario_novo['usuarioId'] = novo_usuario
  return pd.concat([notas, notas_do_usuario_novo])

In [11]:
notas = novo_usuario(usuario_novo_dados)

'O Id do novo usuário é: 612'

In [12]:
sugere_para(612)

,nota_media_dos_usuarios,nota_aparicoes_nos_usuarios,titulo,generos
filmeId,,,,
356,4.571429,7,Forrest Gump (1994),Comedy|Drama|Romance|War
318,4.500000,7,"Shawshank Redemption, The (1994)",Crime|Drama
589,4.428571,7,Terminator 2: Judgment Day (1991),Action|Sci-Fi
110,4.250000,8,Braveheart (1995),Action|Drama|War
364,4.250000,8,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX
...,...,...,...,...
435,2.900000,5,Coneheads (1993),Comedy|Sci-Fi
762,2.700000,5,Striptease (1996),Comedy|Crime
39,2.600000,5,Clueless (1995),Comedy|Romance
